In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

def login_spotify(driver, username, password):
    driver.get('https://accounts.spotify.com/en/login')
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "login-username"))).send_keys(username)
    driver.find_element(By.ID, "login-password").send_keys(password)
    driver.find_element(By.ID, "login-button").click()
    time.sleep(3)

def download_chart_csv(driver, url, download_xpath):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, download_xpath))).click()
    time.sleep(3)

def generate_weekly_dates(start_date: datetime, years_back: int = 5): #차트 날짜 기준
    end_date = start_date - timedelta(days=(years_back * 365))
    dates = []
    while start_date >= end_date:
        dates.append(start_date.strftime("%Y-%m-%d"))
        start_date -= timedelta(weeks=1)
    return dates

def main(username, password, countries, download_xpath_template):
    driver = init_driver()
    login_spotify(driver, username, password)

    dates = generate_weekly_dates(datetime.today())

    for date in dates:
        for country in countries:
            url = f"https://spotifycharts.com/regional/{country}/daily/{date}"
            download_chart_csv(driver, url, download_xpath_template)

    driver.quit()

if __name__ == "__main__":
    USERNAME = "your_username"
    PASSWORD = "your_password"
    COUNTRIES = ['global', 'us', 'kr', 'jp']  # 원하는 국가 코드 추가
    DOWNLOAD_XPATH = "//*[@id='content']/div/div/div/span/button"

    main(USERNAME, PASSWORD, COUNTRIES, DOWNLOAD_XPATH)
